<a href="https://colab.research.google.com/github/Naoki0424/my_document/blob/main/20211112_%E5%AE%9A%E6%9C%9F%E5%AE%9F%E8%A1%8C.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# 指定したチャンネルの登録者数推移を計測するスクリプト

## 事前準備

In [ ]:
from apiclient.discovery import build
from apiclient.errors import HttpError
import pandas as pd
from google.colab import drive

In [ ]:
# Youtube data api を利用する上での情報を定義する
DEVELOPER_KEY = ''
YOUTUBE_API_SERVICE_NAME = 'youtube'
YOUTUBE_API_VERSION = 'v3'
 
youtube = build(
    YOUTUBE_API_SERVICE_NAME, 
    YOUTUBE_API_VERSION,
    developerKey=DEVELOPER_KEY
    )

In [ ]:
# ドライブのマウント
drive.mount('/content/gdrive')

Drive already mounted at /content/gdrive; to attempt to forcibly remount, call drive.mount("/content/gdrive", force_remount=True).


In [ ]:
# CSV読み込み
main_csv = pd.read_csv('/content/gdrive/My Drive/Colab Notebooks/work/data/youtube_chanel - youtube_chanel.csv')

In [ ]:
main_csv

,チャンネルID,チャンネル名,開始日,投稿本数\n（2021/11/13）,チャンネル登録者数\n（2021/11/13）
0,UC67Wr_9pA4I0glIxDt_Cpyw,両学長 リベラルアーツ大学,2018-10-05,1197,1610000
1,UCWdFb_w3JI1BPTiBCXERg2Q,AK in カナダ｜AK-English,2016-05-11,148,544000
2,UCfMIdADo6FQayQCOkLYGhrQ,JAXA | 宇宙航空研究開発機構,2009-02-27,1852,299000
3,UC4zNRgdhLb7wczrMcfdrsVQ,JIN [仁],2012-11-27,1241,273000
4,UCw3RCydc5_K74047gunkeaw,れのれらTV / RenoRera TV,2015-02-21,1787,176000
...,...,...,...,...,...
223,UCkxwe9vIUgINTNx0IVBvjfQ,れん Vlog,2021-10-25,2,0
224,UC1WU0XTfP5FsP5kKCBKAX0Q,マサ 勉強vlog,2020-10-02,1,0
225,UC8UO4zHS-fLGkfJ5pVghFyQ,sun vlog,2021-02-24,1,0
226,UC4PxPGdB-JjIrRf88Dx6JEA,あやま ブログ,2021-08-07,3,0


## 情報取得

In [ ]:
# 対象のチャンネルの詳細情報を取得する
def get_video_detail(channel_id):
    search_response = youtube.channels().list(
        part='snippet,statistics',
        id=channel_id,
        maxResults=50
    ).execute()

    return search_response

In [ ]:
# 読み込んだCSVから詳細情報を取得する
video_detail = []

for i in range(len(main_csv)):
  video_detail = video_detail + get_video_detail(main_csv.iloc[i][0])['items']

In [ ]:
import datetime

now = datetime.datetime.now()
now = now.strftime("%Y/%m/%d")
print(now)

2021/11/13


In [ ]:
list = []

column1 = '投稿本数({})'.format(now)
column2 = 'チャンネル登録者数({})'.format(now)

print(len(video_detail))
for result in video_detail:
  list.append({column1: result['statistics']['videoCount'], column2: result['statistics']['subscriberCount']})

df = pd.DataFrame(list)

228


In [ ]:
df.head()

,投稿本数(2021/11/13),チャンネル登録者数(2021/11/13)
0,1197,1610000
1,148,544000
2,1852,299000
3,1241,273000
4,1787,176000


In [ ]:
pd.concat([main_csv, df], axis=1)

,チャンネルID,チャンネル名,開始日,投稿本数\n（2021/11/13）,チャンネル登録者数\n（2021/11/13）,投稿本数(2021/11/13),チャンネル登録者数(2021/11/13)
0,UC67Wr_9pA4I0glIxDt_Cpyw,両学長 リベラルアーツ大学,2018-10-05,1197,1610000,1197,1610000
1,UCWdFb_w3JI1BPTiBCXERg2Q,AK in カナダ｜AK-English,2016-05-11,148,544000,148,544000
2,UCfMIdADo6FQayQCOkLYGhrQ,JAXA | 宇宙航空研究開発機構,2009-02-27,1852,299000,1852,299000
3,UC4zNRgdhLb7wczrMcfdrsVQ,JIN [仁],2012-11-27,1241,273000,1241,273000
4,UCw3RCydc5_K74047gunkeaw,れのれらTV / RenoRera TV,2015-02-21,1787,176000,1787,176000
...,...,...,...,...,...,...,...
223,UCkxwe9vIUgINTNx0IVBvjfQ,れん Vlog,2021-10-25,2,0,2,0
224,UC1WU0XTfP5FsP5kKCBKAX0Q,マサ 勉強vlog,2020-10-02,1,0,1,0
225,UC8UO4zHS-fLGkfJ5pVghFyQ,sun vlog,2021-02-24,1,0,1,0
226,UC4PxPGdB-JjIrRf88Dx6JEA,あやま ブログ,2021-08-07,3,0,3,0


In [ ]:
result.to_csv('/content/gdrive/My Drive/Colab Notebooks/work/data/youtube_chanel - youtube_chanel.csv')

AttributeError: ignored